In [52]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [53]:
import pandas as pd
import numpy as np
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jeopardy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [65]:
nltk.word_tokenize("He isn't a moronic bastard.")

['He', 'is', "n't", 'a', 'moronic', 'bastard', '.']

In [54]:
def preprocess(q):
    #if q is not a string, return empty string
    if not isinstance(q, str):
        return ''
    q = q.lower()
    nltk_tokens = nltk.word_tokenize(q)
    return nltk_tokens

In [55]:
#read csv file
df = pd.read_csv('questions.csv')
# df['question1'] = df['question1'].apply(preprocess)

In [56]:
questions = df.set_index('qid1')['question1'].to_dict()
questions.update(df.set_index('qid2')['question2'].to_dict())
#preprocess the questions
questions = {k: preprocess(v) for k, v in questions.items()}

In [57]:
#make a new dataframe with duplicate questions while dropping question1 and question2 columns
df = df.drop(['question1', 'question2'], axis=1)
duplicate_questions = df[df['is_duplicate'] == 1]
unique_questions = df[df['is_duplicate'] == 0]

RATIO_DUP_UNIQUE = len(duplicate_questions) / len(unique_questions)
print("Ratio of duplicate questions to unique questions: {}".format(RATIO_DUP_UNIQUE))

Ratio of duplicate questions to unique questions: 0.5854104177694133


In [58]:
#split dataset into 70-20-10 while keeping the ratio of duplicate questions to unique questions
train, validate, test = np.split(duplicate_questions.sample(frac=1), [int(.7*len(duplicate_questions)), int(.9*len(duplicate_questions))])
train = pd.concat([train, unique_questions.sample(frac=0.7)], ignore_index=True)
validate = pd.concat([validate, unique_questions.sample(frac=0.2)], ignore_index=True)
test = pd.concat([test, unique_questions.sample(frac=0.1)], ignore_index=True)

In [59]:
train_duplicate_questions = train[train['is_duplicate'] == 1]
train_unique_questions = train[train['is_duplicate'] == 0]
print("Ratio of duplicate questions to unique questions in train set: {}".format(len(train_duplicate_questions) / len(train_unique_questions)))

Ratio of duplicate questions to unique questions in train set: 0.5854076580108888


In [60]:
test_duplicate_questions = test[test['is_duplicate'] == 1]
test_unique_questions = test[test['is_duplicate'] == 0]
print("Ratio of duplicate questions to unique questions in test set: {}".format(len(test_duplicate_questions) / len(test_unique_questions)))

Ratio of duplicate questions to unique questions in test set: 0.5854375784190715


In [61]:
validate_duplicate_questions = validate[validate['is_duplicate'] == 1]
validate_unique_questions = validate[validate['is_duplicate'] == 0]

print("Ratio of duplicate questions to unique questions in validate set: {}".format(len(validate_duplicate_questions) / len(validate_unique_questions)))

Ratio of duplicate questions to unique questions in validate set: 0.5854064968927052
